In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Consolidated_Total_COVID_List.csv")

num_patients = 300
data = data.loc[0:num_patients-1,:]


In [3]:
total_col_list = list(data.columns.values)

#have to drop unnecessary columns
total_col_list.remove('Study ID')

#sample numerical column list --- have to add more column names
num_col_list = ['Days prior to onset of SX', 'Temperature at admission', 'Cr at admmsion', 'Peak Cr during hospital stay', 'Cr at discharge']

#sample dates column list
dates_col_list = ['Date of Admission', 'Date of Death/Discharge']
#what to do with missing date values?
# temp = data.dropna(subset=dates_col_list)

#fetching remaining column names as categorical column names
cat_col_list = []
for i in range(len(total_col_list)):
    if ( not (total_col_list[i] in num_col_list or total_col_list[i] in dates_col_list )):
        cat_col_list.append(total_col_list[i])


In [4]:
#One hot encoding 
def one_hot_encode(data, col_list):
    
    encoded_df = data[col_list[0]]
    for i in range(len(col_list)):
        each_col_df = data[col_list[i]]
        each_col_df = each_col_df.astype(str).str.lower()
        each_col_df = pd.get_dummies(each_col_df, prefix=col_list[i] +"_is")
        encoded_df = pd.concat([encoded_df, each_col_df], axis=1)
    encoded_df = encoded_df.drop(columns=[col_list[0]])
    return encoded_df
    

In [5]:
encoded_df = one_hot_encode(data, cat_col_list)

In [ ]:
from sklearn.impute import KNNImputer
def handle_numeric_missing_data(data, num_col_list):
    numeric_df = data[num_col_list]
    
    #find missing ratio 
    num_df_missing = numeric_df.isna()
    num_df_missing = num_df_missing.sum()
    num_df_missing = num_df_missing/num_patients
    
    #if missing ratio exceeds threshold, drop them
    threshold = 0.90
    drop_col_list = []
    rem_col_list = []
    for i in range(len(num_col_list)):
        if(num_df_missing[num_col_list[i]] > threshold):
           drop_col_list.append(num_col_list[i])
        else:
           rem_col_list.append(num_col_list[i])
    numeric_df = numeric_df.drop(columns=drop_col_list)
    
    #imputation using kNN
    imputer = KNNImputer(n_neighbors=9)
    df_filled = imputer.fit_transform(numeric_df)
    numeric_df = pd.DataFrame(df_filled, columns=rem_col_list)
    return numeric_df

In [ ]:
numeric_df = handle_numeric_missing_data(data, num_col_list)
total_data = pd.concat([numeric_df, encoded_df], axis=1)